In [4]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mplhep as hep
from iminuit import Minuit
from iminuit.cost import LeastSquares
import pickle
import uproot as ur
import re

In [35]:
with open("../../genIII_data/JLab/Ptrg/Ptrg1.pkl", 'rb') as file:
    ped = pickle.load(file)
with open("geometry.pkl", 'rb') as file:
    geometry = pickle.load(file)

input_file = f"../../genIII_data/BNL/CosmicBNL.root"

events = ur.open("%s:raw"%(input_file))

In [37]:
beam = {}

for ch_key in events.keys():
    match = re.match(r"ch_(\d+)", ch_key)
    if not match:
        continue

    ID = int(match.group(1))
    values = np.array(events[ch_key])  # (n_events, 2) — each row is (LG, HG)

    # Initialize lists
    if f"{ID}_HG" not in beam:
        beam[f"{ID}_HG"] = []

    for i, (LG, HG) in enumerate(values):
        beam[f"{ID}_HG"].append(float(HG))

In [38]:
num_channels = 64 * 6
num_events = len(next(iter(beam.values())))

# Compute the true channel number
geometry["true_channel"] = geometry["caen"] * 64 + geometry["caen_channel"]

# Set index to true_channel for quick lookup
df_positions = geometry.set_index("true_channel")

# Precompute valid channels that exist in both beam and geometry
valid_channels = [
    ch for ch in range(num_channels)
    if f"{ch}_HG" in beam and ch in df_positions.index
]
valid_channels = np.array(valid_channels)

ped_means = np.array([ped['HGPedMean'][ch] for ch in valid_channels])
ped_sigmas = np.array([ped['HGPedSigma'][ch] for ch in valid_channels])

# Stack x, y, z into one array
positions = np.stack([x_vals, y_vals, z_vals, layers, chs], axis=1)

beam_hg_mips = []

# User-configurable settings
range1 = np.arange(0, 24)    # First channel range
range2 = np.arange(25, 49)   # Second channel range
required_hits_range1 = 1     # Required hits in range 0–23
threshold = 3             # threshold

for evt in range(num_events):
    if evt % 10000 == 0:
        print(100 * evt / num_events)

    raw_vals = np.array([beam[f"{ch}_HG"][evt] for ch in valid_channels])
    val_subtracted = raw_vals - 130

    # Compute mask for valid hits
    valid_mask = (
        (val_subtracted >= threshold * 70)
    )

    # Compute MIPs where valid, else 0
    mips_vals = np.zeros_like(val_subtracted)
    mips_vals[valid_mask] = val_subtracted[valid_mask]

    # Get indices for each channel range in the valid_channels array
    idx_range1 = np.where(np.isin(valid_channels, range1))[0]
    idx_range2 = np.where(np.isin(valid_channels, range2))[0]

    # Count how many channels in each range exceed the threshold
    hits_range1 = np.sum(mips_vals[idx_range1] >= threshold)
    hits_range2 = np.sum(mips_vals[idx_range2] >= threshold)

    # Filter based on the required number of hits in each range
    #if hits_range1 == required_hits_range1:
    beam_hg_mips.append(mips_vals)



beam_hg_mips = np.array(beam_hg_mips, dtype=object)


0.0


In [42]:
with open('../../genIII_data/BNL/BNL_Cosmic.pkl', 'wb') as f:
    pickle.dump(beam_hg_mips, f)

In [43]:
print(beam_data)

{'positions': array([[-85.4  , -85.4  ,   0.   ,   0.   ,   0.   ],
       [-36.6  , -85.4  ,   0.   ,   0.   ,   1.   ],
       [ 12.2  , -85.4  ,   0.   ,   0.   ,   2.   ],
       ...,
       [ 12.2  , 109.8  , 379.568,  14.   ,  22.   ],
       [ 61.   , 109.8  , 379.568,  14.   ,  23.   ],
       [109.8  , 109.8  , 379.568,  14.   ,  24.   ]]), 'events': array([[0.0, 0.0, 0.0, ..., 7970.0, 8035.0, 8021.0],
       [0.0, 0.0, 0.0, ..., 0.0, 293.0, 652.0],
       [0.0, 0.0, 0.0, ..., 0.0, 0.0, 292.0],
       ...,
       [0.0, 0.0, 0.0, ..., 0.0, 230.0, 566.0],
       [0.0, 0.0, 0.0, ..., 0.0, 0.0, 258.0],
       [0.0, 0.0, 0.0, ..., 0.0, 0.0, 512.0]], dtype=object)}
